In [ ]:
import pandas as pd
import numpy as np
import random
import os

def get_expected_score(ra, rb):
    return 1 / (1 + 10 ** ((rb - ra) / 400))

def update_elo(ra, rb, result, k=32):
    expected_a = get_expected_score(ra, rb)
    return ra + k * (result - expected_a), rb + k * ((1 - result) - (1 - expected_a))

def run_interactive_ranking(elo_df, titles, star_rating):
    bucket = elo_df[elo_df['original_rating'] == star_rating].index.tolist()
    if len(bucket) < 2: return elo_df

    print(f"--- Ranking {star_rating}-Star Books ({len(bucket)} books) ---")
    while True:
        try:
            # Weighted random choice favoring fewer matches
            weights = 1 / (elo_df.loc[bucket, 'matches_played'] + 1)
            idx_a, idx_b = random.choices(bucket, weights=weights, k=2)
            if idx_a == idx_b: continue
            
            choice = input(f"[1]{titles.get(elo_df.at[idx_a, 'book_id'], 'Unknown')} > [2]{titles.get(elo_df.at[idx_b, 'book_id'], 'Unknown')}").strip().lower()

            if choice == 'q': break
            if choice not in ['1', '2']: continue

            # Update State
            elo_df.loc[[idx_a, idx_b], 'matches_played'] += 1
            ra, rb = elo_df.at[idx_a, 'elo_score'], elo_df.at[idx_b, 'elo_score']
            new_ra, new_rb = update_elo(ra, rb, 1 if choice == '1' else 0)
            elo_df.at[idx_a, 'elo_score'], elo_df.at[idx_b, 'elo_score'] = new_ra, new_rb
            
        except KeyboardInterrupt:
            break
    return elo_df

def compute_continuous(elo_df):
    """Normalizes Elo within star buckets to 0.0-0.99 range."""
    results = pd.Series(index=elo_df['book_id'], dtype=float)
    for stars in elo_df['original_rating'].dropna().unique():
        mask = elo_df['original_rating'] == stars
        subset = elo_df.loc[mask, 'elo_score']

        target_ids = elo_df.loc[mask, 'book_id']

        if len(subset) > 1 and subset.max() > subset.min():
            norm = (subset - subset.min()) / (subset.max() - subset.min())
            results.loc[target_ids] = stars + (norm * 0.99).values
        else:
            results.loc[target_ids] = stars + 0.5
    return results

def refine_ratings(target_df, rating_col, title_col='title', elo_path='data/elo_ratings.csv'):
    target_clean = target_df.dropna(subset=[rating_col]).copy()
    if os.path.exists(elo_path):
        elo_df = pd.read_csv(elo_path)
    else:
        elo_df = pd.DataFrame(columns=['book_id', 'original_rating', 'elo_score', 'matches_played'])
    elo_df = elo_df.set_index('book_id')
    
    common_books = elo_df.index.intersection(target_clean['book_id'])
    elo_df.loc[common_books, 'original_rating'] = target_clean.set_index('book_id').loc[common_books, rating_col]

    # Add new books
    new_books = target_clean[~target_clean['book_id'].isin(elo_df.index)]
    new_entries = pd.DataFrame({
        'original_rating': new_books[rating_col].values,
        'elo_score': 1200.0,
        'matches_played': 0
    }, index=new_books['book_id'])
    elo_df = pd.concat([elo_df, new_entries]).reset_index().rename(columns={'index': 'book_id'})

    # 3. Interactive Ranking
    if input(f"Rank '{rating_col}' buckets? (y/n): ").lower() == 'y':
        titles = dict(zip(target_df['book_id'], target_df[title_col]))
        for star in sorted(target_clean[rating_col].unique(), reverse=True):
            elo_df = run_interactive_ranking(elo_df, titles, star)
        elo_df.to_csv(elo_path, index=False)

    return compute_continuous(elo_df)

books_df = pd.read_csv("data/books.csv")
books_df['description'] = books_df['description'].fillna('').astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()
books_df['lang'] = books_df['lang'].apply(lambda x: "|".join(i.strip() for i in x.split(";")) if isinstance(x, str) else x)

gr_export = pd.read_csv('data/goodreads_library_export.csv')
gr_export['my_rating'] = gr_export['my_rating'].astype('UInt8').replace(0, np.nan)

my_refined = refine_ratings(gr_export, 'my_rating')
books_df = books_df.merge(my_refined.rename('my_rating'), left_on='book_id', right_index=True, how='left')


In [ ]:
# Prep embedding strings
def format_string_for_embedding(items, kind=None, truncate=0):
    if not isinstance(items, (list)) or len(items) == 0:
        return ""

    n = len(items)
    if n == 1:
        res = items[0]
    elif n > truncate > 1:
        res = f"{', '.join(items[:truncate])}, and {items[truncate]}"
    else:
        res = f"{', '.join(items[:-1])}{',' if n > 2 else ''} and {items[-1]}"
    
    prefix = f"{kind.capitalize()}{'s' if n > 1 else ''}: " if kind else ""
    return f"{prefix}{res}"

books_df['authors_post'] = books_df['authors'].str.split('|')
books_df['authors_post'] = books_df['authors_post'].apply(lambda x:format_string_for_embedding(x, truncate=4))

books_df['genres_post'] = books_df['genres'].str.split('|')
books_df['genres_post'] = books_df['genres_post'].apply(lambda x:format_string_for_embedding(x, kind='genre'))

books_df['desc_post'] = [[desc] if isinstance(desc, str) else [] for desc in books_df['description']]
books_df['desc_post'] = books_df['desc_post'].apply(lambda x:format_string_for_embedding(x, kind='description'))

def join_embedding_parts(title, authors, genres, desc):
    text = f"Book: {title}\n"
    if authors:
        text += f"Written by: {authors}\n"
    if genres:
        text += f"{genres}\n"
    if desc:
        text += f"{desc}" 
    return text

books_df['embedding_input'] = [
    join_embedding_parts(t, a, g, d) 
    for t, a, g, d in zip(books_df['title'], books_df['authors_post'], books_df['genres_post'], books_df['desc_post'])
]
id_to_string = books_df.set_index('book_id')['embedding_input']

In [ ]:
# Embed sentences
import os
import ollama
from tqdm import tqdm
import torch
from sklearn.preprocessing import Normalizer

PARAMS = 8
OLLAMA_MODEL = f"qwen3-embedding:{PARAMS}b"
MIN_DIMENSIONS = 32

embeddings_path = f'data/{PARAMS}b_embeddings.csv'
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
if os.path.exists(embeddings_path):
    embeddings = pd.read_csv(embeddings_path, index_col='book_id')
    embedding_cols = embeddings.columns.astype(int)
    loaded_book_ids = embeddings.index.values
    embeddings = torch.tensor(embeddings.values, dtype=torch.float32, device=device)
else:
    loaded_book_ids = np.array([])
    embeddings = None

current_book_ids = books_df['book_id'].values
missing_mask = ~np.isin(current_book_ids, loaded_book_ids)
missing_ids = current_book_ids[missing_mask]
if len(missing_ids) > 0:
    missing_strings = id_to_string.loc[missing_ids].tolist()
    batch_size = 128
    new_embeddings = []
    for i in tqdm(range(0, len(missing_strings), batch_size)):
        batch = missing_strings[i : i + batch_size]
        response = ollama.embed(model=OLLAMA_MODEL, input=batch)
        batch_embeddings = torch.tensor(response['embeddings'], dtype=torch.float32, device=device)
        new_embeddings.append(batch_embeddings)

    new_embeddings = torch.cat(new_embeddings, dim=0)
    if embeddings is not None:
        dim = max(embeddings.shape[1], new_embeddings.shape[1])
        if embeddings.shape[1] < dim:
            pad = torch.zeros((embeddings.shape[0], dim - embeddings.shape[1]), device=device)
            embeddings = torch.cat([embeddings, pad], dim=1)
        
        if new_embeddings.shape[1] < dim:
            pad = torch.zeros((new_embeddings.shape[0], dim - new_embeddings.shape[1]), device=device)
            new_embeddings = torch.cat([new_embeddings, pad], dim=1)

        full_embeddings = torch.empty((len(current_book_ids), dim), dtype=torch.float32, device=device)
        existing_idx_map = {book_id: pos for pos, book_id in enumerate(loaded_book_ids)}
        for i, book_id in enumerate(current_book_ids):
            if book_id in existing_idx_map:
                old_pos = existing_idx_map[book_id]
                full_embeddings[i] = embeddings[old_pos]
        
        missing_positions = np.where(missing_mask)[0]
        full_embeddings[missing_positions] = new_embeddings
    else:
        full_embeddings = new_embeddings

    save_df = pd.DataFrame(full_embeddings.cpu().numpy(), index=current_book_ids)
    save_df.index.name = 'book_id'
    save_df.to_csv(embeddings_path)

    embeddings = full_embeddings
    del full_embeddings, new_embeddings, save_df, batch_embeddings

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

def get_similar_friend_ratings(books_df, friends_path, embeddings, z_score=2):  
    friends = pd.read_csv(friends_path)
    friends['rating'] = friends['rating'].astype('UInt8').replace(0, np.nan)
    friends['list_id'] = friends['list_id'].astype(str)

    friends_pivot = friends.pivot(index='book_id', columns='list_id', values='rating')
    common_book_ratings = friends_pivot.merge(
        books_df[['book_id', 'my_rating']].set_index('book_id'),
        left_index=True,
        right_index=True,
        how='outer'
    )
    rating_columns = [col for col in common_book_ratings.columns if col != 'book_id']
    common_book_ratings = common_book_ratings.dropna(subset=rating_columns, how='all')
    common_book_ratings = common_book_ratings.loc[[id for id in common_book_ratings.index if id in current_book_ids]]
    common_book_ratings = common_book_ratings.dropna(axis='columns', how='all')

    common_book_ids = common_book_ratings.index.values
    book_id_to_global_idx = {book_id: i for i, book_id in enumerate(books_df['book_id'])}
    common_global_indices = torch.tensor([book_id_to_global_idx[bid] for bid in common_book_ids], device=device)
    common_embeddings = embeddings[common_global_indices].to(device)

    # Predict missing ratings to reduce following correlation sparsity
    for list_id in common_book_ratings.columns:
        books_rated = ~(common_book_ratings[list_id].isna())
        y_train = common_book_ratings[books_rated][list_id]

        mask_array = books_rated.values
        X_train = common_embeddings[mask_array].cpu().numpy()
        y_train = y_train.values

        knn = KNeighborsRegressor(
            metric='cosine',
            weights='distance',
            n_jobs=-1
            )
        knn.fit(X_train, y_train)
        pred_rating = knn.predict(common_embeddings.cpu().numpy())
        common_book_ratings[list_id] = pred_rating

    corr_matrix = common_book_ratings.corr(method='pearson')
    np.fill_diagonal(corr_matrix.values, np.nan)
    mean_corr = corr_matrix.mean().mean()
    std_corr = corr_matrix.std().mean()

    similar_tastes = corr_matrix[corr_matrix['my_rating'] >= mean_corr + std_corr * z_score].index
    similar_tastes = [p for p in similar_tastes if p != 'my_rating']

    similar_friend_ratings = friends[friends['list_id'].isin(similar_tastes)]
    return similar_friend_ratings.groupby('book_id')['rating'].mean().dropna()

similar_friend_ratings = get_similar_friend_ratings(books_df, "data/friend_ratings.csv", embeddings, z_score=2)
books_df['training_ratings'] = books_df['my_rating'].fillna(books_df['book_id'].map(similar_friend_ratings))
books_df

In [ ]:
# # MRL dim reduction
train_size = (~books_df['training_ratings'].isna()).sum()
mrl_dimensions = MIN_DIMENSIONS
while mrl_dimensions*2 < train_size:
    mrl_dimensions*=2

mrl_dimensions

# embeddings = embeddings[:, :mrl_dimensions*2] # qwen-3 embedding supports MRL
# norm = Normalizer(norm='l2')
# embeddings = norm.fit_transform(embeddings)


# embeddings = embeddings.loc[books_df['book_id']].values

# embeddings = embeddings[:, :mrl_dimensions*2] # qwen-3 embedding supports MRL
# norm = Normalizer(norm='l2')
# embeddings = norm.fit_transform(embeddings)

# embeddings = torch.tensor(embeddings, dtype=torch.float32)

In [ ]:
# Build adjacency matrix
from torch_geometric.utils import add_self_loops
from torch_geometric.nn.conv.gcn_conv import gcn_norm

id_to_idx = {id: i for i, id in enumerate(books_df['book_id'])}

edge_indices = []
for idx, row in tqdm(books_df.iterrows(), total=len(books_df)):
    current_idx = id_to_idx[row['book_id']]
    if pd.isna(row['similar_books']):
        continue
    for item in row['similar_books'].split('|'):
        try:
            target_id = int(item.split(':')[0])
            if target_id in books_df['book_id'].values:
                target_idx = id_to_idx[target_id]
                edge_indices.append([current_idx, target_idx])
                edge_indices.append([target_idx, current_idx])
        except (ValueError, IndexError):
            continue

if not edge_indices:
    edge_index = torch.tensor([[], []], dtype=torch.long)
else:
    edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

edge_index_with_loops, _ = add_self_loops(edge_index, num_nodes=embeddings.size(0))
edge_index_norm, edge_weight_norm = gcn_norm(edge_index_with_loops, num_nodes=embeddings.size(0))

edge_index_norm = edge_index_norm
edge_weight_norm = edge_weight_norm

adj_matrix = torch.sparse_coo_tensor(edge_index_norm, edge_weight_norm, (embeddings.size(0), embeddings.size(0)))

In [ ]:
import nevergrad as ng
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.metrics import ndcg_score, mean_squared_error
from scipy.stats import spearmanr

def objective(num_propagations, 
              knn_neighbors,
              brr_alpha_1, brr_alpha_2, brr_lambda_1, brr_lambda_2, brr_uncertainty_penalty,
              svr_C, svr_epsilon,
              knn_weight, brr_weight
              ):
    
    all_embeddings = precomputed_embeddings[num_propagations]
    X = all_embeddings[training_mask]
    y = my_ratings

    y_trues = []
    y_preds = []
    skf = KFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, test_idx in skf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        knn = KNeighborsRegressor(
            n_neighbors=knn_neighbors,
            metric='cosine', 
            weights='distance',
            n_jobs=-1,
            )
        knn.fit(X_train, y_train)
        knn_pred = knn.predict(X_test)

        brr = BayesianRidge(
            alpha_1=brr_alpha_1,
            alpha_2=brr_alpha_2,
            lambda_1=brr_lambda_1,
            lambda_2=brr_lambda_2,
            compute_score=True
            )
        brr.fit(X_train, y_train)
        brr_mu, brr_std = brr.predict(X_test, return_std=True)
        brr_pred = brr_mu - (brr_uncertainty_penalty * brr_std)

        svr = SVR(
            kernel='rbf',
            gamma='scale',
            C=svr_C, 
            epsilon=svr_epsilon
            ) 
        svr.fit(X_train, y_train)
        svr_pred = svr.predict(X_test)

        remaining_weight = 1 - knn_weight
        brr_weight *= remaining_weight
        svr_weight = remaining_weight - brr_weight
        final_pred = (knn_weight * knn_pred) + (brr_weight * brr_pred) + (svr_weight * svr_pred)

        y_trues.append(y_test)
        y_preds.append(final_pred)

    y_trues = np.concatenate(y_trues)
    y_preds = np.concatenate(y_preds)
    
    mse = mean_squared_error(y_trues, y_preds)
    ndcg = ndcg_score([y_trues], [y_preds])
    if np.std(y_preds) < 1e-9:
        spearman = 0.0
    else:
        spearman, _ = spearmanr(y_trues, y_preds)
        if np.isnan(spearman): spearman = 0.0

    return mse + (1.0 - ndcg) + (1.0 - spearman)
    # return ndcg - spearman

MAX_PROPAGATIONS = 3
propagated = embeddings.clone()
norm_l2 = Normalizer(norm='l2')
precomputed_embeddings = [norm_l2.transform(propagated.numpy())]

for _ in range(MAX_PROPAGATIONS):
    propagated = torch.sparse.mm(adj_matrix, propagated)
    precomputed_embeddings.append(norm_l2.transform(propagated.numpy()))
del propagated

training_mask = ~books_df['training_ratings'].isna()
my_ratings = books_df.loc[training_mask, 'training_ratings'].values

parametrization = ng.p.Instrumentation(
    num_propagations = ng.p.Scalar(lower=0, upper=MAX_PROPAGATIONS).set_integer_casting(),

    knn_neighbors = ng.p.Scalar(lower=1, upper=mrl_dimensions//3).set_integer_casting(),
    brr_alpha_1=ng.p.Scalar(lower=1e-7, upper=1e-3),
    brr_alpha_2=ng.p.Scalar(lower=1e-7, upper=1e-3),
    brr_lambda_1=ng.p.Log(lower=1e-6, upper=1e-1),
    brr_lambda_2=ng.p.Log(lower=1e-6, upper=1e-1),
    brr_uncertainty_penalty=ng.p.Scalar(lower=0, upper=2.0),
    svr_C = ng.p.Log(lower=1e-3, upper=1e2),
    svr_epsilon = ng.p.Scalar(lower=0.0, upper=1.0),

    knn_weight=ng.p.Scalar(lower=0, upper=1), 
    brr_weight=ng.p.Scalar(lower=0, upper=1), 
)

BUDGET = 300
optimizer = ng.optimizers.NGOpt(parametrization=parametrization, budget=BUDGET)
best_loss = float('inf')
with tqdm(total=BUDGET) as pbar:
    for i in range(BUDGET):
        x = optimizer.ask()
        loss = objective(*x.args, **x.kwargs)
        optimizer.tell(x, loss)
        if loss < best_loss:
            best_loss = loss
        pbar.update(1)
        if i % 10 == 0:
            pbar.set_description(f"Best Loss: {best_loss:.4f}")

best_params = optimizer.provide_recommendation().kwargs

In [ ]:
"""8 MRL sliced
Best Loss: 1.5714: 100%|██████████| 300/300 [11:49<00:00,  2.37s/it]
{'num_propagations': 0,
 'knn_neighbors': 17,
 'brr_alpha_1': 0.0006421576956932416,
 'brr_alpha_2': 0.0004509358497670652,
 'brr_lambda_1': 8.24637589893822e-06,
 'brr_lambda_2': 0.0001968595842801841,
 'brr_uncertainty_penalty': 1.3679433872638418,
 'svr_C': 0.004026523432673821,
 'svr_epsilon': 0.3118081208045217,
 'knn_weight': 0.5805897543886746,
 'brr_weight': 0.7793002684038448}"""

"""8 full
Best Loss: 1.6065: 100%|██████████| 300/300 [16:57<00:00,  3.39s/it]
 {'num_propagations': 0,
 'knn_neighbors': 36,
 'brr_alpha_1': 0.0006880608281295565,
 'brr_alpha_2': 0.0003014766980578955,
 'brr_lambda_1': 1.3770534826263817e-05,
 'brr_lambda_2': 0.005746291033999427,
 'brr_uncertainty_penalty': 1.8570930538472121,
 'svr_C': 0.03383255010384741,
 'svr_epsilon': 0.44291277801367485,
 'knn_weight': 0.5848794696920626,
 'brr_weight': 0.5411997331122358}"""

"""0.6 MRL sliced
Best Loss: 1.5748: 100%|██████████| 300/300 [10:29<00:00,  2.10s/it]
{'num_propagations': 0,
 'knn_neighbors': 37,
 'brr_alpha_1': 0.0002978589454364143,
 'brr_alpha_2': 0.0005679816409316726,
 'brr_lambda_1': 0.0003994602308099561,
 'brr_lambda_2': 0.003659493431608586,
 'brr_uncertainty_penalty': 1.3409360588302992,
 'svr_C': 0.009413204523151376,
 'svr_epsilon': 0.5861865063902226,
 'knn_weight': 0.6849389456290866,
 'brr_weight': 0.7766436222538672}"""

"""0.6 full
Best Loss: 1.5926: 100%|██████████| 300/300 [12:22<00:00,  2.48s/it]
{'num_propagations': 0,
 'knn_neighbors': 55,
 'brr_alpha_1': 0.0007517429857762481,
 'brr_alpha_2': 0.0003941767071639492,
 'brr_lambda_1': 8.604754211823616e-06,
 'brr_lambda_2': 0.008083208576838542,
 'brr_uncertainty_penalty': 1.048016018935488,
 'svr_C': 0.013400702917698843,
 'svr_epsilon': 0.4149283525703265,
 'knn_weight': 0.6826121601964188,
 'brr_weight': 0.8414845392757134}"""
best_params

In [ ]:
# Run optimized hyperparams
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor

all_embeddings = precomputed_embeddings[best_params['num_propagations']]

X_train = all_embeddings[training_mask]
y_train = my_ratings

knn = KNeighborsRegressor(
    n_neighbors=best_params['knn_neighbors'],
    metric='cosine', 
    weights='distance',
    n_jobs=-1,
    )
knn.fit(X_train, y_train)
knn_pred = knn.predict(all_embeddings)

brr = BayesianRidge(
    alpha_1=best_params['brr_alpha_1'],
    alpha_2=best_params['brr_alpha_2'],
    lambda_1=best_params['brr_lambda_1'],
    lambda_2=best_params['brr_lambda_2'],
    compute_score=True
    )
brr.fit(X_train, y_train)
brr_mu, brr_std = brr.predict(all_embeddings, return_std=True)
brr_pred = brr_mu - (best_params['brr_uncertainty_penalty'] * brr_std)

svr = SVR(
    kernel='rbf',
    gamma='scale',
    C=best_params['svr_C'], 
    epsilon=best_params['svr_epsilon']
    ) 
svr.fit(X_train, y_train)
svr_pred = svr.predict(all_embeddings)

knn_weight = best_params['knn_weight']
brr_weight = best_params['brr_weight']

remaining_weight = 1 - knn_weight
brr_weight *= remaining_weight
svr_weight = remaining_weight - brr_weight
final_pred = (knn_weight * knn_pred) + (brr_weight * brr_pred) + (svr_weight * svr_pred)

books_df['knn_pred'] = knn_pred
books_df['brr_pred'] = brr_pred
books_df['svr_pred'] = svr_pred
books_df['final_pred'] = final_pred

# cols = ['title'] + [col for col in books_df.columns if col.endswith('pred')]
# books_df[books_df['my_rating'].isna()].sort_values(by='final_pred', ascending=False)

In [ ]:
# Count adjusted rating
C = books_df['avg_rating'].mean()
m = books_df['rating_count'].quantile(0.10) 
def weighted_rating(x, m=m, C=C):
    v = float(x['rating_count'])
    R = float(x['avg_rating'])
    if v == 0: 
        return C
    return (v / (v + m) * R) + (m / (v + m) * C)
books_df['count_adjusted_rating'] = books_df.apply(weighted_rating, axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
books_df[['count_adjusted_rating', 'final_pred']] = scaler.fit_transform(books_df[['count_adjusted_rating', 'final_pred']])

# # Quadratic model coefficients
# from sklearn.preprocessing import MinMaxScaler

# def fit_quadratic(row):
#     x = np.array([1, 2, 3, 4, 5])
#     a, b, c = np.polyfit(x, row, 2)
#     return pd.Series([a, b, c])

# books_df['1_star_percentage'] = books_df['1_star'] / books_df['rating_count']
# books_df['2_star_percentage'] = books_df['2_star'] / books_df['rating_count']
# books_df['3_star_percentage'] = books_df['3_star'] / books_df['rating_count']
# books_df['4_star_percentage'] = books_df['4_star'] / books_df['rating_count']
# books_df['5_star_percentage'] = books_df['5_star'] / books_df['rating_count']
# coefficients = books_df[['1_star_percentage','2_star_percentage','3_star_percentage','4_star_percentage','5_star_percentage']].apply(fit_quadratic, axis=1)
# books_df.drop(['1_star_percentage','2_star_percentage','3_star_percentage','4_star_percentage','5_star_percentage'], axis=1, inplace=True)

# books_df['a'], books_df['b'], books_df['c'] = coefficients[0], coefficients[1], coefficients[2]
# scaler = MinMaxScaler()
# books_df[['a', 'b', 'c']] = scaler.fit_transform(books_df[['a', 'b', 'c']]) + 1
# books_df['coeff_rating'] = books_df['a'] * books_df['c']
# books_df['final_rating'] = books_df[['coeff_rating']] * books_df['count_adjusted_rating'] * books_df['final_pred'] / np.log1p(books_df['num_pages'])

In [ ]:
books_df['final_rating'] = (books_df['count_adjusted_rating'] * books_df['final_pred']) #/ (np.log1p(books_df['num_pages']))
books_df[books_df['my_rating'].isna()].sort_values(by='final_rating', ascending=False)

In [ ]:
books_df['final_rating'] = (books_df['count_adjusted_rating'] * books_df['final_pred']) #/ (np.log1p(books_df['num_pages']))
books_df[books_df['my_rating'].isna()].sort_values(by='final_rating', ascending=False)

In [ ]:
# Types of rankings
books_df['num_adjusted_rating'] = books_df['average_rating'] - (books_df['average_rating'] - books_df['average_rating'].mean()) / books_df['num_ratings_ln']
books_df['coeff_2a_rating'] = (books_df['num_adjusted_rating'] * books_df['2a_shifted'])
books_df['coeff_b_rating'] = (books_df['num_adjusted_rating']) / (books_df['b_shifted'])
books_df['coeff_c_rating'] = (books_df['num_adjusted_rating'] * books_df['c_shifted'])
books_df['joined_rating'] = (books_df['num_adjusted_rating'] * books_df['c_shifted'] * books_df['2a_shifted']) / books_df['b_shifted']
books_df['final_rating'] = books_df['joined_rating'] - (books_df['joined_rating'] - books_df['joined_rating'].mean()) / books_df['num_ratings_ln']

books_df['num_adjusted_page_rating'] = books_df['num_adjusted_rating'] / (books_df['num_pages_ln'])
books_df['coeff_2a_page_rating'] = books_df['coeff_2a_rating'] / books_df['num_pages_ln']
books_df['coeff_b_page_rating'] = books_df['coeff_b_rating'] / books_df['num_pages_ln']
books_df['coeff_c_page_rating'] = books_df['coeff_c_rating'] / books_df['num_pages_ln']
books_df['joined_page_rating'] = books_df['joined_rating'] / books_df['num_pages_ln']
books_df['final_page_rating'] = books_df['joined_page_rating'] - (books_df['joined_page_rating'] - books_df['joined_page_rating'].mean()) / books_df['num_ratings_ln']

# old

In [ ]:
goodreads_export = pd.read_csv('data/goodreads_library_export.csv')
# goodreads_export = pd.read_csv('data/20-01-2025_goodreads_library_export.csv')
# goodreads_export = goodreads_export[['Book Id', 'Author', 'My Rating', 'Number of Pages', 'Original Publication Year']]
# goodreads_export = goodreads_export.rename(columns={'Book Id':'book_id',
#                                                     'Author': 'author',
#                                                     'My Rating': 'my_rating',
#                                                     'Number of Pages': 'num_pages',
#                                                     'Original Publication Year': 'year'})
threshold = (goodreads_export['number_of_pages'].mean() - goodreads_export['number_of_pages'].std())
goodreads_export.loc[goodreads_export['number_of_pages'] < threshold, 'number_of_pages'] = np.nan

book_df = pd.read_csv('data/books.csv')
# book_df = pd.read_csv('data/01-2025_goodreads_scraped.csv')
df = goodreads_export.merge(book_df, on='book_id')

# Drop competing columns
df['author'] = df['author_x'].fillna(df['author_y'])
df['num_pages'] = df['num_pages_x'].fillna(df['num_pages_y'])
df['year'] = df['year_x'].fillna(df['year_y'])
df.drop(columns=['author_x', 'author_y', 'num_pages_x', 'num_pages_y', 'year_x', 'year_y'], inplace=True)

df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
df['year'] = df['year'].fillna(df['year'].mean()).round().astype(int)
df['num_pages'] = df['num_pages'].fillna(df['num_pages'].mean()).round().astype(int)
df['num_reviews'] = df['num_reviews'].fillna(0).round().astype(int)
df['my_rating'] = df['my_rating'].replace(0,np.nan)

df['age'] = int(datetime.now().strftime('%Y')) - df['year']
df['average_rating'] = ((df['5 stars'] * 5) + (df['4 stars'] * 4) + (df['3 stars'] * 3) + (df['2 stars'] * 2) + df['1 star']) / df['num_ratings']
df = df[['book_id', 'title', 'author', 'year', 'age', 'series', 'num_pages', 'genres', 'num_ratings', 'num_reviews', 'my_rating', 'average_rating', '5 stars', '4 stars', '3 stars', '2 stars', '1 star']]

In [ ]:
def fit_quadratic(row):
    x = np.array([1, 2, 3, 4, 5])
    a, b, c = np.polyfit(x, row, 2)
    return pd.Series([a, b, c])

# Calculating quadrdic modeling coefficients
df['1_star_percentage'] = df['1 star'] / df['num_ratings']
df['2_star_percentage'] = df['2 stars'] / df['num_ratings']
df['3_star_percentage'] = df['3 stars'] / df['num_ratings']
df['4_star_percentage'] = df['4 stars'] / df['num_ratings']
df['5_star_percentage'] = df['5 stars'] / df['num_ratings']
coefficients = df[['1_star_percentage','2_star_percentage','3_star_percentage','4_star_percentage','5_star_percentage']].apply(fit_quadratic, axis=1)
df['a'], df['b'], df['c'] = coefficients[0], coefficients[1], coefficients[2]

# Pre-processing columns for rankings
df['num_ratings_ln'] = np.log1p(df['num_ratings'])
df['num_pages_ln'] = np.log1p(df['num_pages'])
df['2a_shifted'] = df['a'] - df['a'].min()
df['2a_shifted'] = df['2a_shifted'] * (1 / df['2a_shifted'].max()) + 1
df['b_shifted'] = df['b'] - df['b'].min()
df['b_shifted'] = df['b_shifted'] * (1 / df['b_shifted'].max()) + 1
df['c_shifted'] = df['c'] - df['c'].min()
df['c_shifted'] = df['c_shifted'] * (1 / df['c_shifted'].max()) + 1

# Types of rankings
df['num_adjusted_rating'] = df['average_rating'] - (df['average_rating'] - df['average_rating'].mean()) / df['num_ratings_ln']
df['coeff_2a_rating'] = (df['num_adjusted_rating'] * df['2a_shifted'])
df['coeff_b_rating'] = (df['num_adjusted_rating']) / (df['b_shifted'])
df['coeff_c_rating'] = (df['num_adjusted_rating'] * df['c_shifted'])
df['joined_rating'] = (df['num_adjusted_rating'] * df['c_shifted'] * df['2a_shifted']) / df['b_shifted']
df['final_rating'] = df['joined_rating'] - (df['joined_rating'] - df['joined_rating'].mean()) / df['num_ratings_ln']

df['num_adjusted_page_rating'] = df['num_adjusted_rating'] / (df['num_pages_ln'])
df['coeff_2a_page_rating'] = df['coeff_2a_rating'] / df['num_pages_ln']
df['coeff_b_page_rating'] = df['coeff_b_rating'] / df['num_pages_ln']
df['coeff_c_page_rating'] = df['coeff_c_rating'] / df['num_pages_ln']
df['joined_page_rating'] = df['joined_rating'] / df['num_pages_ln']
df['final_page_rating'] = df['joined_page_rating'] - (df['joined_page_rating'] - df['joined_page_rating'].mean()) / df['num_ratings_ln']

In [ ]:
numeric_cols = ['age', 'num_pages', 'num_pages_ln', 'num_ratings', 'num_ratings_ln', 'num_reviews', 'my_rating', 'average_rating', '1 star', '2 stars', '3 stars', '4 stars', '5 stars', '1_star_percentage', '2_star_percentage', '3_star_percentage', '4_star_percentage', '5_star_percentage', 'a', 'b', 'c', 'num_adjusted_rating', 'coeff_2a_rating', 'coeff_b_rating', 'coeff_c_rating', 'joined_rating', 'final_rating', 'num_adjusted_page_rating', 'coeff_2a_page_rating', 'coeff_b_page_rating', 'coeff_c_page_rating', 'joined_page_rating', 'final_page_rating']
corr_df= df[numeric_cols].corr()

plt.figure(figsize=(20, 15)) 
sns.heatmap(corr_df, annot=True, cmap='coolwarm', linewidths=0.5) 
plt.title('Correlation Heatmap') 
plt.show()

In [ ]:
fresh = df.sort_values(by='final_page_rating', ascending=False).reset_index().drop('index', axis=1)
fresh = fresh[fresh['my_rating'].isna()]
fresh[['Fiction' in genre_list for genre_list in fresh['genres']]] # Fiction, Nonfiction, Memoir, Classics, History, Politics, Philosophy, Business